In [14]:
import os
import json
from datetime import datetime
from tqdm.notebook import tqdm
import fbpyutils
from fbpyutils import file as fu, image as uu

env, logger = fbpyutils.get_env(), fbpyutils.get_logger()

In [15]:
logger.info(f"Running using AppConfig: {env.APP}")

2025-07-20 10:56:06,264 - fbpyutils - INFO - Running using AppConfig: name='FBPyUtils' version='1.6.6' environment='dev' appcode='FBPYUTILS' year=2025


In [ ]:
inclusion_list = (
    ("I:/", "*", "YGGDRASIL_DISK_IA"),
    (r"G:\Meu Drive", "*", "YGGDRASIL_GDRIVE"),
    (r"C:\Users\fcjbispo", "*", "YGGDRASIL_FCBISPO_HOME"),
    (r"E:\Shared", "*", "YGGDRASIL_DATA_SHARED")
)

exclusion_list = (
    f"{os.path.sep}.", 
    f"{os.path.sep}AppData{os.path.sep}", 
    f"{os.path.sep}CrossDevice{os.path.sep}", 
)

report_target_path=r"C:\Users\fcjbispo\Downloads"

# source_dir, source_mask, source_label =

files_on_source = fu.find(source_dir, source_mask, recurse=True, parallel=True)

logger.info(f"Found {len(files_on_source)} files on {source_label} ({source_dir})")

2025-07-20 11:06:02,651 - fbpyutils - INFO - Default multi-threads parallelization available
2025-07-20 11:06:02,653 - fbpyutils - INFO - Process initialized: parallel=True, type=threads, workers=16
2025-07-20 11:06:02,654 - fbpyutils - INFO - Starting execution with 3 parameter sets.
2025-07-20 11:06:02,655 - fbpyutils - INFO - Starting parallel execution with 16 workers using threads.
2025-07-20 11:06:04,113 - fbpyutils - INFO - Processed 3 items successfully in parallel.
2025-07-20 11:06:04,272 - fbpyutils - INFO - Found 6105 files on DATA_SHARED (E:\Shared)


In [44]:
files_to_process = [f for f in files_on_source if f not in [f for x in exclusion_list if f.__contains__(x)]]
logger.info(f"Filtered {len(files_to_process)} files on {source_label} ({source_dir}) to process")

2025-07-20 11:06:04,306 - fbpyutils - INFO - Filtered 4067 files on DATA_SHARED (E:\Shared) to process


In [45]:
report = {
    "source_name": source_label,
    "source_dir": source_dir,
    "source_mask": source_mask,
    "start_extraction": datetime.now(),
    "files": [],
}

file_report = {
    "file_path": "",
    "status": "OK", 
    "status_message": None
}

for f in tqdm(files_to_process):
    file_report["file_path"] = f
    try:
        file_report["details"] = fu.describe_file(f)
        file_report["status"] = "OK"
        file_report["status_message"] = None
    except Exception as e:
        file_report["details"] = None
        file_report["status"] = "ERROR"
        file_report["status_message"] = str(e)
    report["files"].append(file_report.copy())

report["end_extraction"] = datetime.now()

  0%|          | 0/4067 [00:00<?, ?it/s]

2025-07-20 11:08:25,171 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR\Local\DCIM\Camera\Thumbs.db'. Defaulting to 'application/octet-stream'.
2025-07-20 11:08:33,768 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR\Local\Download\BoardingPass.pkpass'. Defaulting to 'application/octet-stream'.
2025-07-20 11:08:34,287 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR\Local\Download\FREYA-FCJBISPO.rdp'. Defaulting to 'application/octet-stream'.
2025-07-20 11:08:35,014 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR\Local\Download\Precificação do Azure Data Factory'. Defaulting to 'application/octet-stream'.
2025-07-20 11:08:35,977 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR\Local\Download\Tata Code of Conduct 2015 - Contents'. Defaulting to 'application/octet-stream'.
2025-07-20 11:08:36,001 - fbpyutils - WARNING - Could not guess mime type for 'E:\Shared\MJOLNIR

In [46]:
report.keys(),len(report['files'])

(dict_keys(['source_name', 'source_dir', 'source_mask', 'start_extraction', 'files', 'end_extraction']),
 4067)

In [47]:
report['end_extraction']

datetime.datetime(2025, 7, 20, 11, 8, 46, 557791)

In [48]:
report_target_path=os.path.sep.join([r"C:\Users\fcjbispo\Downloads", f"REPORT_{source_label}_{report['end_extraction'].strftime('%Y%m%d%H%M%S')}.json"])

In [51]:
with open(report_target_path, "w", encoding="utf-8") as f:
    f.write(json.dumps(report, ensure_ascii=False, allow_nan=True, default=str, sort_keys=False, indent=4))
    print(f"Report for {source_label} writed at {report_target_path}.")

Report for DATA_SHARED writed at C:\Users\fcjbispo\Downloads\REPORT_DATA_SHARED_20250720110846.json.
